In [25]:
from scipy import stats
import scipy
import feather as ft
import os
import pandas as pd
import numpy as np
import time
from collections import Counter
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import datetime as dt
import configparser
import re
import utils
import seaborn as sns
from matplotlib_venn import venn3
pd.options.display.max_rows = 20
pd.options.display.max_columns = 50
import warnings
warnings.filterwarnings('ignore')

In [ ]:
1h - 90s

In [26]:
path = "../../share/data_prod/"
game_daily_summary = ft.read_dataframe(path +'game_daily_summary.ft').reset_index(drop=True)
member_summary = ft.read_dataframe(path +'member_summary.ft').reset_index(drop=True)

In [30]:
end = pd.datetime.today().date()
start = end - pd.Timedelta('1 days')
login_today = ft.read_dataframe(path + 'member_login_type.ft')

In [31]:
bet_today = game_daily_summary.query('date >= @start & date < @end')
dep_today = member_summary.query('Date >= @start & Date < @end')

In [32]:
# 判斷要顯示的偏好遊戲
bet_today_member_product = bet_today.groupby(['member_name', 'product_type'])[
    'gross_revenue_payout_time'].sum().reset_index().sort_values(by=['member_name', 'gross_revenue_payout_time'],
                                                                 ascending=True)
# 要顯示的投注额、盈利总额、盈利率
bet_today_member = bet_today.groupby(['member_name', 'member_id'])[
    'gross_revenue_payout_time', 'stake'].sum().reset_index()
bet_today_member['盈利率'] = bet_today_member.gross_revenue_payout_time / bet_today_member.stake
bet_today_member = bet_today_member.sort_values(by='gross_revenue_payout_time', ascending=True)
bet_top15 = bet_today_member.head(15).reset_index(drop=True)
bet_top15['gross_revenue_payout_time'] = bet_top15['gross_revenue_payout_time'] * -1

bet_today_member_product = bet_today_member_product[bet_today_member_product.member_name.isin(bet_top15.member_name)]
rank = []
name = ''
for index in bet_today_member_product.index:
    now_name = bet_today_member_product.loc[index, 'member_name']
    if now_name == name:
        rank_now += 1
        rank.append(rank_now)
    else:
        rank_now = 1
        rank.append(rank_now)
        name = now_name
bet_today_member_product['rank'] = rank
bet_today_member_product = bet_today_member_product.query('rank == 1')[['member_name', 'product_type']]

login_today = login_today[login_today.MemberId.isin(bet_top15.member_id)][['MemberId', 'IPLocation']]
# 找出登入區域
login_today['用户地区'] = login_today.IPLocation.apply(lambda x: find_city(x))
dep_today_target = dep_today[dep_today.MemberName.isin(bet_top15.member_name)]
dep_today_target = dep_today_target.groupby('MemberName').DepositAmount.sum().reset_index().rename(
    columns={'MemberName': 'member_name'})
bet_top15 = bet_top15.merge(login_today[['MemberId', '用户地区']].rename(columns={'MemberId': 'member_id'}), on='member_id',
                            how='left')
bet_top15 = bet_top15.merge(bet_today_member_product, on='member_name', how='left')
bet_top15 = bet_top15.merge(dep_today_target, on='member_name', how='left')
bet_top15.DepositAmount = bet_top15.DepositAmount.fillna(0)
bet_top15 = bet_top15.merge(member[['MemberName', 'o_member_name']].rename(columns={'MemberName': 'member_name'}),
                            on='member_name', how='left')
bet_top15['original_name'] = bet_top15['o_member_name']
bet_top15['昨日排名'] = range(1, 16)
bet_top15 = bet_top15[['昨日排名', 'original_name', 'o_member_name', '用户地区', 'product_type', 'DepositAmount', 'stake',
                       'gross_revenue_payout_time', '盈利率']].rename(
    columns={'o_member_name': '用户名', 'product_type': '游戏类型', 'DepositAmount': '存款总额', 'stake': '投注额',
             'gross_revenue_payout_time': '盈利总额'})
bet_top15.用户名 = bet_top15.用户名.apply(lambda x: x[:2] + '**' + x[-2:])
bet_top15['加码礼金'] = [1077, 777, 577, 377, 377, 177, 177, 77, 77, 77, 0, 0, 0, 0, 0]
for col_name in ['存款总额', '投注额', '盈利总额', '加码礼金']:
    bet_top15[col_name] = bet_top15[col_name].apply(lambda x: '{:20,.2f}'.format(x)[:-3])
bet_top15['盈利率'] = bet_top15['盈利率'].apply(lambda x: str(round(x * 100 * -1)) + '%')
bet_top15.游戏类型 = bet_top15.游戏类型.apply(lambda x: trans_name(x))

ValueError: You are trying to merge on float64 and object columns. If you wish to proceed you should use pd.concat

In [33]:
bet_top15

,gross_revenue_payout_time,stake,盈利率,member_id,用户地区,member_name,product_type


In [12]:
path = "../../share/data_prod/"
game_daily_summary = ft.read_dataframe(path +'game_daily_summary.ft').reset_index(drop=True)
# 昨日
query_time_end = pd.datetime.today().date()
query_time_start = query_time_end - pd.Timedelta('1 days')
# 目標遊戲
target_game_type = ['IMVGCard', 'IMKYCard', 'IMLYCard']

# 資料篩選
tmp = game_daily_summary.query('date >= @query_time_start & date < @query_time_end & product_type.isin(@target_game_type)').groupby(['member_name', 'member_id']).stake.sum().reset_index().sort_values(by = 'stake').reset_index()

# 反水計算
tmp['rebate'] = tmp.stake*0.01
tmp.loc[tmp.rebate > 10000, 'rebate'] = 10000

In [24]:
tmp

,index,member_name,member_id,stake,rebate
0,16,244307001,456681,3.00,0.0300
1,138,qwe5581,155371,3.20,0.0320
2,170,wegame7620,456563,6.00,0.0600
3,97,lihao111333,233487,14.00,0.1400
4,36,a1102362521,464281,14.60,0.1460
5,174,win588,49319,14.70,0.1470
6,199,yang1187,165513,28.50,0.2850
7,195,xiuxiu1,482877,34.60,0.3460
8,151,stone0851,333039,39.20,0.3920
9,162,waj2003,13272,40.00,0.4000
